# ECE 498 DS Mini Project 2
avk2@illinois.edu 

zinan2@illinois.edu

jg8@illinois.edu

## Task 0

### Process pcap files into Pandas Dataframe

In [4]:
import pandas as pd
import pyshark
import json

In [5]:
dns = pyshark.FileCapture('./data/dns.pcap', keep_packets=True)

In [6]:
http = pyshark.FileCapture('./data/http.pcap', keep_packets=True)

In [7]:
http2 = pyshark.FileCapture('./data/http2.pcap', keep_packets=True)

In [9]:
!pip install nest_asyncio

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import nest_asyncio
nest_asyncio.apply()

##### dns

In [11]:
dns_data = []
for pkt in dns:
    d = {
        'sniff_time': pkt.sniff_time,
        'length': pkt.length,
        'highest_layer': pkt.highest_layer,
        'time_stamp': pkt.sniff_timestamp,
        'ip': pkt.ip
#         'port': pkt[2].port
    }
    dns_data.append(d)

In [12]:
df_dns = pd.DataFrame(dns_data, columns = ['sniff_time', 'length', 'highest_layer', 'time_stamp', 'ip'])
df_dns.to_csv("dns_df.csv", index=False)

##### http

In [19]:
http_data = []

for pkt in http:
    for i in range(len(pkt.layers)):
        if pkt[i].layer_name == 'http':        
            try:
                d = {
                    'sniff_time': pkt.sniff_time,
                    'length': pkt.length,
                    'highest_layer': pkt.highest_layer,
                    'time_stamp': pkt.sniff_timestamp,
                    'ip': pkt.ip.dst,
                    'port': pkt.tcp.port,
                    'content_type': pkt.http.content_type
                }
            except AttributeError:
                pass
            
    http_data.append(d)

In [20]:
df_http = pd.DataFrame(http_data, columns = ['sniff_time', 'length', 'highest_layer', 'time_stamp', 'ip', 'port', 'content_type'])
df_http.to_csv("http_df.csv", index=False)

##### http2

In [21]:
http2_data = []

for pkt in http2:
    for i in range(len(pkt.layers)):
        if pkt[i].layer_name == 'http':        
            try:
                d = {
                    'sniff_time': pkt.sniff_time,
                    'length': pkt.length,
                    'highest_layer': pkt.highest_layer,
                    'time_stamp': pkt.sniff_timestamp,
                    'ip': pkt.ip.dst,
                    'port': pkt.tcp.port,
                    'content_type': pkt.http.content_type
                }
            except AttributeError:
                pass
            
    http2_data.append(d)

In [22]:
df_http2 = pd.DataFrame(http2_data, columns = ['sniff_time', 'length', 'highest_layer', 'time_stamp', 'ip', 'port', 'content_type'])
df_http2.to_csv("http2_df.csv", index=False)

### Process Operating System Query logs into Pandas Dataframe

In [24]:
log_list = []
with open("./data/osqueryd.results.log", "r") as log_data:
    for line in log_data:
        log_list.append(eval(line))
df_log = pd.json_normalize(log_list)

In [25]:
df_log.to_csv("oslogs.csv", index=False)

### Which one is attacker

In [27]:
http_cmd = 0

for pck in http:
    try:
        content_type = pck.http.Content_type
        if '#cmd' in content_type:
            http_cmd += 1
    
    except AttributeError:
        pass

In [28]:
http2_cmd = 0

for pck in http2:
    try:
        content_type = pck.http.Content_type
        if '#cmd' in content_type:
            http2_cmd += 1
    
    except AttributeError:
        pass

In [29]:
print(f'#cmd in http content-type is {http_cmd}, #cmd in http2 content-type is {http2_cmd}')

#cmd in http content-type is 7, #cmd in http2 content-type is 0


The http pcap file represents attacker activity.  It contains HTTP layers that have #cmd strings in their content type.  The legitimate http2 file has no such HTTP layers and no evidence of #cmd strings.

# Task 1

In [13]:
http[30].http.request_uri

'/showcase.action'